# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

12:00:27 | NewsletterAgent.newsletter_agent | INFO | Test info message
12:00:27 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
12:00:27 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [5]:
print("🚀 Creating NewsletterAgent...")

do_download=True
process_since=None
# process_since='2025-10-05 18:30:00'

try:
    # set up state
    session_id = 'test_newsletter_20251010093252452130'
    step_name = 'step_07_select_sections'
#     del session_id, step_name
except Exception as e:
    print(e)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

12:00:28 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Using provided state with 117 articles


🚀 Creating NewsletterAgent...
session_id is defined
test_newsletter_20251010093252452130
Initialized NewsletterAgent with persistent state and 9-step workflow
Session ID: test_newsletter_20251010093252452130


In [8]:
agent.state.get_status()


{'headlines': {'total': 0},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_01_fetch_urls',
  'workflow_complete': False,
  'workflow_status': 'not_started',
  'workflow_status_message': '',
  'progress_percentage': 0.0,
  'max_edits': 2,
  'concurrency': 16,
  'do_download': True,
  'process_since': None},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [9]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


09:33:01 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Starting check_workflow_status
09:33:01 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Completed check_workflow_status


⏱️  Total execution time: 11.78s
📊 Final result:
Current workflow status:

- Progress: 0.0% (0/9 complete)
- 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Fetch Urls

Step-by-step status:
- Step 1: Fetch Urls — not_started
- Step 2: Filter Urls — not_started
- Step 3: Download Articles — not_started
- Step 4: Extract Summaries — not_started
- Step 5: Rate Articles — not_started
- Step 6: Cluster By Topic — not_started
- Step 7: Select Sections — not_started
- Step 8: Draft Sections — not_started
- Step 9: Finalize Newsletter — not_started

Would you like me to start Step 1 (gather URLs) or resume/run all steps?


In [10]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


09:33:13 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Starting check_workflow_status
09:33:13 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Completed check_workflow_status
09:33:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Starting Step 1: Gather URLs
09:33:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Cleaning download/sources: 
09:33:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Successfully cleaned download/sources
09:33:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Cleaning download/text: 
09:33:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Successfully cleaned download/text
09:33:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Cleaning download/html: 
09:33:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Successfully cleaned download/html
2025-10-10 09:33:14,941 - fetcher_5906269776 - INFO - [fetcher_in

▶ Starting Step 1: step_01_fetch_urls


2025-10-10 09:33:15,204 - fetcher_5906269776 - INFO - [fetch_rss] RSS fetch successful for HackerNoon: 50 articles
2025-10-10 09:33:15,205 - fetcher_5906269776 - INFO - [fetch_html] Fetching HTML from Techmeme: https://www.techmeme.com/river
2025-10-10 09:33:15,267 - fetcher_5906269776 - INFO - [fetch_rss] RSS fetch successful for Ars Technica: 20 articles
2025-10-10 09:33:15,269 - fetcher_5906269776 - INFO - [fetch_rss] Fetching RSS from The Register: https://www.theregister.com/software/ai_ml/headlines.atom
2025-10-10 09:33:15,350 - fetcher_5906269776 - INFO - [fetch_rss] RSS fetch successful for Hacker News: 30 articles
2025-10-10 09:33:15,357 - fetcher_5906269776 - INFO - [fetch_html] Fetching HTML from The Verge: https://www.theverge.com/ai-artificial-intelligence
2025-10-10 09:33:15,433 - fetcher_5906269776 - INFO - [fetch_rss] RSS fetch successful for The Register: 50 articles
2025-10-10 09:33:15,436 - fetcher_5906269776 - INFO - [fetch_html] Fetching HTML from VentureBeat: http

2025-10-10 09:33:17,819 - fetcher_5906269776 - INFO - [fetch_html] Source dict for Reddit: {'type': 'html', 'url': 'https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+technews+technology/top/?sort=top&t=day', 'filename': 'Reddit', 'exclude': ['^https://www.reddit.com/user/', '^https://www.reddit.com/domain/', '^https://chat.reddit.com/', '^https://i.redd.it/', '^https://redditblog.com/', '^https://www.redditinc.com/', '^https://www.reddithelp.com/', '^https://itunes.apple.com/', '^https://play.google.com/', '^https?://(?:www\\.)?reddit\\.com/r/([^/]+)/?$', '^https://www.reddit.com/help/', '^https://www.reddit.com/contact/', '^https://www.reddit.com/rules/'], 'scroll': 5, 'minlength': 8, 'rss': 'https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLear

...

2025-10-10 09:33:43,157 - fetcher_5906269776 - DEBUG - Parsed datetime: 2025-10-11 00:33:43
2025-10-10 09:33:43,157 - fetcher_5906269776 - DEBUG - Added UTC timezone: 2025-10-11 00:33:43+00:00
2025-10-10 09:33:43,157 - fetcher_5906269776 - DEBUG - Converted to UTC: 2025-10-11 00:33:43+00:00
2025-10-10 09:33:43,157 - fetcher_5906269776 - DEBUG - Formatted last_updated: 2025-10-11T00:33:43Z
2025-10-10 09:33:43,158 - fetcher_5906269776 - INFO - Saving HTML to download/sources/Reddit.html
2025-10-10 09:33:43,161 - fetcher_5906269776 - INFO - [fetch_html] Parsing HTML file: download/sources/Reddit.html
2025-10-10 09:33:43,188 - fetcher_5906269776 - INFO - [fetch_html] Parsed HTML file: download/sources/Reddit.html
2025-10-10 09:33:43,188 - fetcher_5906269776 - INFO - [fetch_html] HTML fetch successful for Reddit: 61 articles
2025-10-10 09:33:43,330 - fetcher_5906269776 - DEBUG - performed human like actions
2025-10-10 09:33:43,331 - fetcher_5906269776 - INFO - Scrolling Feedly_AI (1/3)
2025

,source,url
0,Ars Technica,20
1,Bloomberg,30
2,Business Insider,16
3,FT,47
4,Feedly AI,66
5,Hacker News,29
6,HackerNoon,50
7,New York Times,24
8,NewsAPI,98
9,Reddit,51


,source,title,url,published,rss_summary,id
0,Ars Technica,“Like putting on glasses for the first time”—h...,https://arstechnica.com/science/2025/10/like-p...,"Fri, 10 Oct 2025 11:00:24 +0000",AI is “comically good” at detecting small eart...,0
1,Ars Technica,AI models can acquire backdoors from surprisin...,https://arstechnica.com/ai/2025/10/ai-models-c...,"Thu, 09 Oct 2025 22:03:21 +0000","Anthropic study suggests ""poison"" training att...",1
2,Ars Technica,Bank of England warns AI stock bubble rivals 2...,https://arstechnica.com/ai/2025/10/bank-of-eng...,"Wed, 08 Oct 2025 21:18:30 +0000",Central bank says market concentration hasn't ...,2
3,Ars Technica,Vandals deface ads for AI necklaces that liste...,https://arstechnica.com/tech-policy/2025/10/va...,"Wed, 08 Oct 2025 16:07:29 +0000",Critics attacked subway ads to defend human fr...,3
4,Ars Technica,Insurers balk at paying out huge settlements f...,https://arstechnica.com/ai/2025/10/insurers-ba...,"Wed, 08 Oct 2025 13:31:28 +0000","OpenAI, Anthropic consider using investor fund...",4
...,...,...,...,...,...,...
769,NewsAPI,Hallmark Channel Stars Issue Warning After Sca...,https://www.today.com/popculture/news/hallmark...,2025-10-09T12:26:49Z,NaN,769
770,NewsAPI,AI Bubble Warning: Financial Institutions Soun...,https://www.huffpost.com/entry/ai-bubble-warni...,2025-10-09T11:59:40Z,NaN,770
771,NewsAPI,Healthcare Contract Management Software Market...,https://www.globenewswire.com/news-release/202...,2025-10-09T09:37:00Z,NaN,771
772,NewsAPI,AI shortens time it takes to measure the susta...,https://techxplore.com/news/2025-10-ai-shorten...,2025-10-09T13:18:05Z,NaN,772


09:33:49 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Completed Step 1: Gathered 878 articles
09:33:54 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Starting check_workflow_status
09:33:54 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Completed check_workflow_status


⏱️  Total execution time: 45.96s
📊 Final result:
Step 1 (Fetch URLs) completed.

Summary:
- Collected 878 headlines from 17 sources via RSS; 774 articles stored in persistent state.
- Workflow progress: 1/9 complete (11.1%).
- Next step: Step 2 — Filter URLs (AI-related only).

Shall I proceed to Step 2 now?


In [11]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,“Like putting on glasses for the first time”—h...,https://arstechnica.com/science/2025/10/like-p...,"Fri, 10 Oct 2025 11:00:24 +0000",AI is “comically good” at detecting small eart...,0
1,Ars Technica,AI models can acquire backdoors from surprisin...,https://arstechnica.com/ai/2025/10/ai-models-c...,"Thu, 09 Oct 2025 22:03:21 +0000","Anthropic study suggests ""poison"" training att...",1
2,Ars Technica,Bank of England warns AI stock bubble rivals 2...,https://arstechnica.com/ai/2025/10/bank-of-eng...,"Wed, 08 Oct 2025 21:18:30 +0000",Central bank says market concentration hasn't ...,2
3,Ars Technica,Vandals deface ads for AI necklaces that liste...,https://arstechnica.com/tech-policy/2025/10/va...,"Wed, 08 Oct 2025 16:07:29 +0000",Critics attacked subway ads to defend human fr...,3
4,Ars Technica,Insurers balk at paying out huge settlements f...,https://arstechnica.com/ai/2025/10/insurers-ba...,"Wed, 08 Oct 2025 13:31:28 +0000","OpenAI, Anthropic consider using investor fund...",4
...,...,...,...,...,...,...
769,NewsAPI,Hallmark Channel Stars Issue Warning After Sca...,https://www.today.com/popculture/news/hallmark...,2025-10-09T12:26:49Z,NaN,769
770,NewsAPI,AI Bubble Warning: Financial Institutions Soun...,https://www.huffpost.com/entry/ai-bubble-warni...,2025-10-09T11:59:40Z,NaN,770
771,NewsAPI,Healthcare Contract Management Software Market...,https://www.globenewswire.com/news-release/202...,2025-10-09T09:37:00Z,NaN,771
772,NewsAPI,AI shortens time it takes to measure the susta...,https://techxplore.com/news/2025-10-ai-shorten...,2025-10-09T13:18:05Z,NaN,772


In [12]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
10,Techmeme,194
8,NewsAPI,98
4,Feedly AI,66
9,Reddit,51
12,The Register,50
6,HackerNoon,50
3,FT,47
1,Bloomberg,30
5,Hacker News,29
13,The Verge,26


In [13]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:33:56 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Starting Step 2: Filter URLs
09:33:56 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | 🔍 Filtering 774 headlines...
09:33:56 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | 🔄 Checking for duplicates (all urls without date restrictions)
09:33:56 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | 🔍 Filtering 774 articles for dupes.


▶ Starting Step 2: step_02_filter_urls
checking https://arstechnica.com/science/2025/10/like-putting-on-glasses-for-the-first-time-how-ai-improves-earthquake-detection/
url not found - inserting new URL
checking https://arstechnica.com/ai/2025/10/ai-models-can-acquire-backdoors-from-surprisingly-few-malicious-documents/
url not found - inserting new URL
checking https://arstechnica.com/ai/2025/10/bank-of-england-warns-ai-stock-bubble-rivals-2000-dotcom-peak/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/tech-policy/2025/10/vandals-deface-ads-for-ai-necklaces-that-listen-to-all-your-conversations/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/insurers-balk-at-paying-out-huge-settlements-for-claims-against-ai-firms/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/ars-live-is-the-ai-bubble-about-to-pop-a-live-chat-with-ed-zitron/
found url, no cutoff set - treati

url not found - inserting new URL
checking https://finance.yahoo.com/news/coursera-inc-cour-chatgpt-integration-071520498.html
url not found - inserting new URL
checking https://www.fastcompany.com/91419058/tesla-robotaxi-bay-area
url not found - inserting new URL
checking https://fortune.com/2025/10/10/pepsico-taps-walmart-veteran-cfo-bets-big-beverage-revamp/
url not found - inserting new URL
checking https://www.technologyreview.com/2025/10/10/1125586/the-download-our-bodies-memories-and-tratons-electric-trucks/
url not found - inserting new URL
checking https://www.digitaltrends.com/phones/googles-ai-now-lets-you-try-on-shoes-without-leaving-home-or-putting-on-socks/
url not found - inserting new URL
checking https://www.cnbc.com/2025/10/10/china-blacklists-major-chip-ai-research-firm-techinsights-analyzed-report-huawei.html
url not found - inserting new URL
checking https://finance.yahoo.com/news/aerovironment-inc-avav-analyst-highlights-071532154.html
url not found - inserting ne

09:33:56 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | URL deduplication: 478 duplicates filtered, 296 new URLs remain
09:33:56 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | 🔍 Filtering 774 headlines for AI relevance using LLM...
INFO:llm:Initialized LangfuseClient


url not found - inserting new URL
checking https://data-star.dev/
url not found - inserting new URL
checking https://vkoskiv.com/first-linux-patch/
found url, no cutoff set - treating as duplicate
checking https://www.rfleury.com/p/multi-core-by-default
url not found - inserting new URL
checking https://theabbie.github.io/DoublePendulum/
url not found - inserting new URL
checking https://www.figure.ai/news/introducing-figure-03
found url, no cutoff set - treating as duplicate
checking https://github.com/mafik/keyer
found url, no cutoff set - treating as duplicate
checking https://penntoday.upenn.edu/news/select-neurons-brainstem-may-hold-key-treating-chronic-pain
found url, no cutoff set - treating as duplicate
checking https://twitter.com/karpathy/status/1976077806443569355
found url, no cutoff set - treating as duplicate
checking https://www.galois.com/articles/automated-lean-proofs-for-every-type
url not found - inserting new URL
checking https://www.subwaybuilder.com/
found url, no

INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=459, user_len=954
09:33:56 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Processing 12 chunks with concurrency 16


concurrency:  16


09:34:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Completed Step 2: 145 AI-related articles, 478 duplicates removed


⏱️  Total execution time: 18.66s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Removed 478 duplicate URLs, classified 296 new articles, found 145 AI-related.


In [14]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

09:34:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Starting Step 3: Download Articles
09:34:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Starting concurrent scraping of 145 AI-related articles


▶ Starting Step 3: step_03_download_articles


09:34:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Launching browser for 145 URLs with 16 concurrent workers
09:34:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Worker 0 fetching 1 of 145 https://arstechnica.com/science/2025/10/like-putting-on-glasses-for-the-first-time-how-ai-improves-earthquake-detection/
09:34:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | scrape_url(https://arstechnica.com/science/2025/10/like-putting-on-glasses-for-the-first-time-how-ai-improves-earthquake-detection/)
09:34:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | scraping https://arstechnica.com/science/2025/10/like-putting-on-glasses-for-the-first-time-how-ai-improves-earthquake-detection/ to download/html
09:34:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Downloading https://arstechnica.com/science/2025/10/like-putting-on-glasses-for-the-first-time-how-ai-improves-earthquake-detection/
09:34:14 | Newsl

09:34:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Downloading https://www.ft.com/content/8d5387f2-62b0-4830-b0e4-00ba0622a7c8
09:34:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Worker 10 fetching 12 of 145 https://www.forbes.com/sites/angelachan/2025/10/09/sandy-sholls-next-act-how-zelig-is-teaching-ai-the-rules-of-fashion/
09:34:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | scrape_url(https://www.forbes.com/sites/angelachan/2025/10/09/sandy-sholls-next-act-how-zelig-is-teaching-ai-the-rules-of-fashion/)
09:34:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | scraping https://www.forbes.com/sites/angelachan/2025/10/09/sandy-sholls-next-act-how-zelig-is-teaching-ai-the-rules-of-fashion/ to download/html
09:34:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Downloading https://www.forbes.com/sites/angelachan/2025/10/09/sandy-sholls-next-act-how-zelig-is-teaching-ai-the-rules-of-fashion/


09:34:50 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | scrape_url(https://www.forbes.com/sites/richardnieva/2025/10/09/amd-openai-chip-software/)
09:34:50 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | scraping https://www.forbes.com/sites/richardnieva/2025/10/09/amd-openai-chip-software/ to download/html
09:34:50 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Downloading https://www.forbes.com/sites/richardnieva/2025/10/09/amd-openai-chip-software/
09:34:51 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Saving HTML to download/html/Global_A.I._Influenced_Cybersecurity_Market_Size_Projected_to_Reach__234_Billion_By_2032.html
09:34:52 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Worker 3 completed https://www.prnewswire.co.uk/news-releases/global-ai-influenced-cybersecurity-market-size-projected-to-reach-234-billion-by-2032-302579253.html with status: 200
09:34:52 | NewsletterAgent.test_newsletter_20

...

concurrency:  16


09:45:41 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Inserted 2 new site records
09:45:41 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Updated 145 URL records with final URLs


Starting with 145 rows...
Processing 145 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Getting embeddings for 135 texts...


09:45:45 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Completed Step 3: Downloaded 135 articles


Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
  Pair: 111 vs 47
    111: NewsAPI - Google's AI Search Lets You Try on Shoes Without Having to Stand, Wear Socks
    47: Feedly AI - Google’s AI now lets you try on shoes without leaving home, or putting on socks
Removing 1 rows due to high similarity 
Final dataframe has 144 rows (removed 1 rows)
⏱️  Total execution time: 691.10s
📊 Final result:
✅ Step 3 step_03_download_articles completed successfully! Downloaded 135 AI-related articles with 94% success rate.
📊 Average article length: 5225 characters
🔗 Content stored in persistent state.


In [15]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["status"]!=200])




,id,source,title,status,final_url
134,137,Feedly AI,Stop worrying about the AI bubble: SocGen strategists tell clients the dollar and fundamentals will sustain this rally.,401,https://www.marketwatch.com/story/stop-worrying-about-the-ai-bubble-socgen-strategists-tell-clients-the-dollar-and-fundamentals-will-sustain-this-rally-b8c25628
135,616,WSJ,"Microsoft, Anthropic Hire Former British Prime Minister Rishi Sunak",skipped,https://www.wsj.com/world/uk/microsoft-anthropic-hire-former-british-prime-minister-rishi-sunak-4677fa73
136,717,NewsAPI,VisionAI India hosts the emerging tech conference 2025 at IIT Delhi,410,https://timesofindia.indiatimes.com/business/india-business/visionai-india-hosts-the-emerging-tech-conference-2025-at-iit-delhi/articleshow/124410756.cms
137,41,Bloomberg,"Japan’s Top Bank CEOs Push for AI, Soothe Worry Over Human Work",skipped,https://www.bloomberg.com/news/articles/2025-10-09/japan-s-top-bank-ceos-push-for-ai-soothe-worry-over-human-work
138,40,Bloomberg,"Samsung Electronics Shares Hit Record High on AI, Chip Optimism",skipped,https://www.bloomberg.com/news/articles/2025-10-10/samsung-shares-on-course-for-record-high-on-ai-chip-optimism
139,32,Bloomberg,CoreWeave Insiders Sold $1 Billion Post-IPO as AI Rich Cash Out,skipped,https://www.bloomberg.com/news/articles/2025-10-10/coreweave-crwv-insiders-sold-1-billion-of-stock-in-q3-as-ai-rich-cash-out
140,27,Bloomberg,"Senate Passes AI Chip Export Limits on Nvidia, AMD to China",skipped,https://www.bloomberg.com/news/articles/2025-10-10/senate-passes-ai-chip-export-limits-on-nvidia-amd-to-china
141,25,Bloomberg,"AI Jobs Shock Is Coming and Firms Aren’t Ready, Klarna CEO Says",skipped,https://www.bloomberg.com/news/articles/2025-10-10/ai-jobs-shock-is-coming-and-firms-aren-t-ready-klarna-ceo-says
142,24,Bloomberg,"AI Will Not Lead to UK Job Losses, Says Bank of England Governor",skipped,https://www.bloomberg.com/news/articles/2025-10-10/ai-will-not-lead-to-uk-job-losses-says-bank-of-england-governor
143,121,Feedly AI,Selling $50 Million Penthouses With a Little Help From AI,skipped,https://www.wsj.com/articles/selling-50-million-penthouses-with-a-little-help-from-ai-f5d8ffd1


In [16]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["html_path"]==""])



,id,source,title,status,final_url
134,137,Feedly AI,Stop worrying about the AI bubble: SocGen strategists tell clients the dollar and fundamentals will sustain this rally.,401,https://www.marketwatch.com/story/stop-worrying-about-the-ai-bubble-socgen-strategists-tell-clients-the-dollar-and-fundamentals-will-sustain-this-rally-b8c25628
135,616,WSJ,"Microsoft, Anthropic Hire Former British Prime Minister Rishi Sunak",skipped,https://www.wsj.com/world/uk/microsoft-anthropic-hire-former-british-prime-minister-rishi-sunak-4677fa73
136,717,NewsAPI,VisionAI India hosts the emerging tech conference 2025 at IIT Delhi,410,https://timesofindia.indiatimes.com/business/india-business/visionai-india-hosts-the-emerging-tech-conference-2025-at-iit-delhi/articleshow/124410756.cms
137,41,Bloomberg,"Japan’s Top Bank CEOs Push for AI, Soothe Worry Over Human Work",skipped,https://www.bloomberg.com/news/articles/2025-10-09/japan-s-top-bank-ceos-push-for-ai-soothe-worry-over-human-work
138,40,Bloomberg,"Samsung Electronics Shares Hit Record High on AI, Chip Optimism",skipped,https://www.bloomberg.com/news/articles/2025-10-10/samsung-shares-on-course-for-record-high-on-ai-chip-optimism
139,32,Bloomberg,CoreWeave Insiders Sold $1 Billion Post-IPO as AI Rich Cash Out,skipped,https://www.bloomberg.com/news/articles/2025-10-10/coreweave-crwv-insiders-sold-1-billion-of-stock-in-q3-as-ai-rich-cash-out
140,27,Bloomberg,"Senate Passes AI Chip Export Limits on Nvidia, AMD to China",skipped,https://www.bloomberg.com/news/articles/2025-10-10/senate-passes-ai-chip-export-limits-on-nvidia-amd-to-china
141,25,Bloomberg,"AI Jobs Shock Is Coming and Firms Aren’t Ready, Klarna CEO Says",skipped,https://www.bloomberg.com/news/articles/2025-10-10/ai-jobs-shock-is-coming-and-firms-aren-t-ready-klarna-ceo-says
142,24,Bloomberg,"AI Will Not Lead to UK Job Losses, Says Bank of England Governor",skipped,https://www.bloomberg.com/news/articles/2025-10-10/ai-will-not-lead-to-uk-job-losses-says-bank-of-england-governor
143,121,Feedly AI,Selling $50 Million Penthouses With a Little Help From AI,skipped,https://www.wsj.com/articles/selling-50-million-penthouses-with-a-little-help-from-ai-f5d8ffd1


In [17]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

09:45:45 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Processing 144 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-4.1-mini, system_len=1273, user_len=43
09:45:45 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Using model 'gpt-4.1-mini' for summarization
09:45:45 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Starting summarization for 144 articles
09:45:45 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Processing 144 chunks with concurrency 16


▶ Starting Step 4: step_04_extract_summaries
concurrency:  16


09:46:10 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Extracting metadata from HTML files for 144 articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/item_distiller' from Langfuse
INFO:llm:Parsed prompt 'newsagent/item_distiller': model=gpt-5-mini, system_len=1365, user_len=151
09:46:15 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Processing 144 chunks with concurrency 16


concurrency:  16
⏱️  Total execution time: 136.79s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 144/144 articles.
💾 Summaries stored in headline DataFrame.


In [18]:
# if we get a refusal, examine prompt and delete rows
bad_stuff =["What\'s the Best Way ",
            "AI could make it eas",
            "AI can design toxic ",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



[20, 20, 20]


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags,short_summary


In [19]:
# remove
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [20]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:48:02 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Rating 144 AI articles using fn_rate_articles
09:48:02 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Calculating article ratings for 144 articles
09:48:02 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Rating recency
09:48:02 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Rating spam probability
09:48:02 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Initialized LangfuseClient
09:48:02 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
09:48:02 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


▶ Starting Step 5: step_05_rate_articles
concurrency:  16


09:48:08 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | low quality articles: {0.0: 128, 1.0: 7, 0.9999723899264568: 2, 0.9859363206911979: 1, 8.315279260963632e-07: 1, 0.999088951739003: 1, 0.008577484930474293: 1, 0.999997064707474: 1, 0.7549149672612132: 1}
09:48:08 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
09:48:08 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1790, user_len=240


concurrency:  16


09:48:18 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | on topic articles: {1.0: 114, 0.9999998063873693: 3, 0.9820137760522224: 2, 0.9999832372181827: 1, 0.999999091165777: 1, 0.9999983759447105: 1, 1.006039279283317e-12: 1, 1.2917800047859961e-12: 1, 0.3208212708736474: 1, 0.0010322309695250382: 1, 0.9999996871837232: 1, 0.006692851264215723: 1, 3.927863545481039e-07: 1, 1.3176514270095466e-10: 1, 3.0988191387218256e-12: 1, 1.3615261080896538e-13: 1, 0.49999999904767284: 1, 0.09534945969074979: 1, 0.9999974223173038: 1, 0.6224593113235651: 1, 0.9525741086376607: 1, 3.775134544279098e-11: 1, 1.3007123989067183e-05: 1, 0.8807969943837407: 1, 0.9993736588086098: 1, 0.0: 1, 1.6918979226151304e-10: 1}
09:48:18 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Rating importance probability
09:48:18 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Successfully retrieved prompt 'newsagent/rate_importance' from Langfuse
09:48:18 | NewsletterAgent.

...

,id,site_name,title,bradley_terry,bt_z
91,91,BGR,"Gemini Enterprise Is Official: Gemini 2.5, Nano Banana, Veo 3, Agents, And More AI Tools For Work",8.593086,1.840983
66,66,Yahoo,NVDA: Nvidia Rises After U.S. Approves Billions In AI Chip Exports To UAE,8.473094,1.815276
95,95,TechXplore,"Google wants right to bundle Gemini AI app with Maps, YouTube",7.984182,1.710531
92,92,Forbes,How AMD’s AI Software Helped It Score The Multi-Billion Dollar OpenAI Deal,7.664334,1.642007
38,38,NBC News,ChatGPT safety systems can be bypassed for weapons instructions,7.573709,1.622592
...,...,...,...,...,...
141,141,MarketWatch,Stop worrying about the AI bubble: SocGen strategists tell clients the dollar and fundamentals will sustain this rally.,-8.888685,-1.904312
138,138,Bored Panda,"I Wanted To Know How 26 Celebrities Who Left Us Too Soon Would Be Now, So I Used A.I. To Get The Answer (New Pics)",-9.343579,-2.001769
135,135,Reddit,AI generated Death Metal,-9.985616,-2.139319
142,142,Times of India,VisionAI India hosts the emerging tech conference 2025 at IIT Delhi,-10.794518,-2.312618


10:10:38 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Top 10 ids: [91, 66, 95, 92, 38, 114, 107, 2, 111, 1]
10:10:38 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | After round 22/29: 
10:10:38 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Number of ranking changes: 39
10:10:38 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Sum of absolute ranking changes: 1694.0 (avg rank chg 11.85)
10:10:38 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | last_two: 10.41958041958042, prev_two: 13.853146853146853
10:10:38 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | ---------------------------------------------------
10:10:38 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Running round 23 of max 29
10:10:38 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | ---------------------------------------------------
10:10:38 | NewsletterAgent.test_newsletter_2025101009325245213

,id,site_name,title,bradley_terry,bt_z
91,91,BGR,"Gemini Enterprise Is Official: Gemini 2.5, Nano Banana, Veo 3, Agents, And More AI Tools For Work",8.619462,1.838010
66,66,Yahoo,NVDA: Nvidia Rises After U.S. Approves Billions In AI Chip Exports To UAE,8.499361,1.812400
95,95,TechXplore,"Google wants right to bundle Gemini AI app with Maps, YouTube",8.010430,1.708140
92,92,Forbes,How AMD’s AI Software Helped It Score The Multi-Billion Dollar OpenAI Deal,7.690995,1.640024
38,38,NBC News,ChatGPT safety systems can be bypassed for weapons instructions,7.599802,1.620578
...,...,...,...,...,...
141,141,MarketWatch,Stop worrying about the AI bubble: SocGen strategists tell clients the dollar and fundamentals will sustain this rally.,-8.934815,-1.905255
138,138,Bored Panda,"I Wanted To Know How 26 Celebrities Who Left Us Too Soon Would Be Now, So I Used A.I. To Get The Answer (New Pics)",-9.398452,-2.004121
135,135,Reddit,AI generated Death Metal,-10.031590,-2.139131
142,142,Times of India,VisionAI India hosts the emerging tech conference 2025 at IIT Delhi,-10.840472,-2.311617


10:11:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Top 10 ids: [91, 66, 95, 92, 38, 114, 107, 2, 111, 1]
10:11:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | After round 23/29: 
10:11:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Number of ranking changes: 31
10:11:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Sum of absolute ranking changes: 1108.0 (avg rank chg 7.75)
10:11:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | last_two: 9.797202797202797, prev_two: 9.664335664335663
10:11:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Increase in avg rank change, stopping
10:11:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Low rated articles: 3
10:11:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | low rated article: I Wanted To Know How 26 Celebrities Who Left Us Too Soon Would Be Now, So I Used A.I. To Get The Answer (New Pics) -

⏱️  Total execution time: 1393.98s
📊 Final result:
✅ Step 5 step_05_rate_articles completed successfully! Rated 140 articles with average rating 6.3/10.
⭐ High quality articles (≥7.0): 56
💾 Ratings stored in persistent state.


In [21]:
headline_df=state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df.sort_values("rating", ascending=False)[['site_name', 'title', 'rating', 'short_summary']])


,site_name,title,rating,short_summary
1,Financial Times,China launches customs crackdown on Nvidia AI chips,12.009596,"China has launched customs inspections on Nvidia AI chips after Beijing ordered tech firms to stop buying U.S. processors, a move aimed at restricting foreign AI technology access that could disrupt Nvidia's supply chain and slow AI hardware to China."
0,The New York Times,ChatGPT’s Platform Play + a Trillion-Dollar GPU Empire + the Queen of Slop,11.896737,"OpenAI's DevDay unveiled in-platform app capabilities to embed internet functions into ChatGPT; OpenAI announced a major GPU partnership with AMD to scale AI infrastructure, while speakers raised concerns about the chaotic quality of AI-generated content."
2,Financial Times,"The UK CMA designates Google with “strategic market status” in online search, but excludes Gemini; Google warns such oversight could slow product launches",11.331610,"The UK Competition and Markets Authority designates Google the first firm with strategic market status in online search under new digital competition laws, excluding Gemini, and orders it to loosen control over search services."
14,Ars Technica,AI models can acquire backdoors from surprisingly few malicious documents,9.972831,"Anthropic finds large language models up to 13 billion parameters can develop backdoors from as few as 250 malicious training documents; poisoning effectiveness stays near-constant with model size, highlighting risks from scraped web data and need for stronger data curation."
5,Bloomberg,"Senate Passes AI Chip Export Limits on Nvidia, AMD to China",9.696824,The U.S. Senate passed restrictions limiting exports of AI chips from Nvidia and AMD to China.
...,...,...,...,...
134,OneFootball,Ex-Man Utd player lands golden contract thanks to ChatGPT,0.662050,"OneFootball reports former Manchester United player Demetri Mitchell used ChatGPT to negotiate his Leyton Orient contract without an agent, helping him understand terms and living costs and avoid typical agent fees."
136,Hacker News,Weave (YC W25) is hiring a founding AI engineer,0.400700,"Weave (YC W25), a profitable startup building software to boost engineering productivity, is hiring a founding AI engineer to work directly with its CTO and CEO to develop AI aimed at making engineers’ work ten times easier."
137,The Economic Times,IIT Guwahati’s Mehta Family School of Data Science and AI celebrates graduation of inaugural batch,0.352580,"IIT Guwahati's Mehta Family School of Data Science and AI graduated its inaugural B.Tech class (launched 2021 with Mehta Family Foundation); 91% of 22 students received placements with Warner Bros Discovery, Google and Microsoft, and enrolled at Carnegie Mellon University."
139,Biztoc,FirstEnergy Looks To Capture Long-Term Growth Amid AI Boom; Offers 4.2% Yield,0.253932,FirstEnergy plans to capture long-term growth from rising AI-driven data-center demand and offers a 4.2% dividend yield; the utility sector ranks 48th of 197 industries amid sustained investor interest.


In [22]:
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"""
{row.rating:.1f} - [{row.title}]({row.url}) - {row.site_name}

{row.short_summary}
    """))
    


12.0 - [China launches customs crackdown on Nvidia AI chips](https://www.ft.com/content/8d5387f2-62b0-4830-b0e4-00ba0622a7c8) - Financial Times

China has launched customs inspections on Nvidia AI chips after Beijing ordered tech firms to stop buying U.S. processors, a move aimed at restricting foreign AI technology access that could disrupt Nvidia's supply chain and slow AI hardware to China.
    


11.9 - [ChatGPT’s Platform Play + a Trillion-Dollar GPU Empire + the Queen of Slop](https://www.nytimes.com/2025/10/10/podcasts/hardfork-openai-devday-amd.html) - The New York Times

OpenAI's DevDay unveiled in-platform app capabilities to embed internet functions into ChatGPT; OpenAI announced a major GPU partnership with AMD to scale AI infrastructure, while speakers raised concerns about the chaotic quality of AI-generated content.
    


11.3 - [The UK CMA designates Google with “strategic market status” in online search, but excludes Gemini; Google warns such oversight could slow product launches](https://www.ft.com/content/8cc9b094-698d-41d8-8399-b10dc3e57e70) - Financial Times

The UK Competition and Markets Authority designates Google the first firm with strategic market status in online search under new digital competition laws, excluding Gemini, and orders it to loosen control over search services.
    


10.0 - [AI models can acquire backdoors from surprisingly few malicious documents](https://arstechnica.com/ai/2025/10/ai-models-can-acquire-backdoors-from-surprisingly-few-malicious-documents/) - Ars Technica

Anthropic finds large language models up to 13 billion parameters can develop backdoors from as few as 250 malicious training documents; poisoning effectiveness stays near-constant with model size, highlighting risks from scraped web data and need for stronger data curation.
    


9.7 - [Senate Passes AI Chip Export Limits on Nvidia, AMD to China](https://www.bloomberg.com/news/articles/2025-10-10/senate-passes-ai-chip-export-limits-on-nvidia-amd-to-china) - Bloomberg

The U.S. Senate passed restrictions limiting exports of AI chips from Nvidia and AMD to China.
    

...


0.7 - [Ex-Man Utd player lands golden contract thanks to ChatGPT ](https://onefootball.com/en/news/ex-man-utd-player-lands-golden-contract-thanks-to-chatgpt-41782176) - OneFootball

OneFootball reports former Manchester United player Demetri Mitchell used ChatGPT to negotiate his Leyton Orient contract without an agent, helping him understand terms and living costs and avoid typical agent fees.
    


0.4 - [Weave (YC W25) is hiring a founding AI engineer](https://www.ycombinator.com/companies/weave-3/jobs/SqFnIFE-founding-ai-engineer) - Hacker News

Weave (YC W25), a profitable startup building software to boost engineering productivity, is hiring a founding AI engineer to work directly with its CTO and CEO to develop AI aimed at making engineers’ work ten times easier.
    


0.4 - [IIT Guwahati’s Mehta Family School of Data Science and AI celebrates graduation of inaugural batch](https://economictimes.indiatimes.com/industry/services/education/iit-guwahatis-mehta-family-school-of-data-science-and-ai-celebrates-graduation-of-inaugural-batch/articleshow/124416109.cms) - The Economic Times

IIT Guwahati's Mehta Family School of Data Science and AI graduated its inaugural B.Tech class (launched 2021 with Mehta Family Foundation); 91% of 22 students received placements with Warner Bros Discovery, Google and Microsoft, and enrolled at Carnegie Mellon University.
    


0.3 - [FirstEnergy Looks To Capture Long-Term Growth Amid AI Boom; Offers 4.2% Yield](https://biztoc.com/x/e93ba18b1b9ea712) - Biztoc

FirstEnergy plans to capture long-term growth from rising AI-driven data-center demand and offers a 4.2% dividend yield; the utility sector ranks 48th of 197 industries amid sustained investor interest.
    


0.1 - [AI generated Death Metal](https://v.redd.it/ix5kmilih4uf1) - Reddit

AI has generated death metal lyrics, band concepts and music—sometimes labeled Genocide metal—sparking humorous and provocative online discussions and Reddit sharing of samples as experiments expanding the genre.
    

In [23]:
# User prompt to run workflow
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


10:11:16 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-4.1-mini, system_len=1100, user_len=80
10:11:16 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Using model 'gpt-4.1-mini' for topic extraction
10:11:16 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Processing 140 articles for topic extraction
10:11:16 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Processing 14 chunks with concurrency 16


▶ Starting Step 6: step_06_cluster_by_topic
concurrency:  16


10:11:21 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Successfully extracted 607 total topics across articles
10:11:21 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Starting canonical topic classification for 101 topics
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/can

INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized Langfu

INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_top

...

=== HDBSCAN Parameters ===
min_cluster_size:   2
min_samples:        2
n_components:       37
=== Clustering Quality Metrics ===
Number of clusters: 19
Noise points: 73 (52.1%)
Average cluster size: 3.5 ± 1.3
Cluster size range: 2 - 7
=== Quality Scores ===
Silhouette Score: 0.273 (higher is better)
Calinski-Harabasz Score: 6.8 (higher is better)
Davies-Bouldin Score: 1.120 (lower is better)
HDBSCAN Validity Index: 0.123
Composite Score: 0.198 (higher is better)

[I 2025-10-10 10:12:13,360] Trial 199 finished with value: -0.19772458465470366 and parameters: {'n_components': 37, 'min_cluster_size': 2, 'min_samples': 2}. Best is trial 120 with value: -0.2482484453368911.

Optimization completed!
Best composite score: 0.2482
Best parameters: {'n_components': 20, 'min_cluster_size': 2, 'min_samples': 2}

=== Results with Best Parameters ===
Reduced dimensions from 3072 to 20
=== Clustering Quality Metrics ===
Number of clusters: 18
Noise points: 65 (46.4%)
Average cluster size: 4.2 ± 2.6
C

10:12:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | 9: AI Impacts Across Society
10:12:14 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | “Like putting on glasses for the first time”—how AI improves earthquake detection (Earthquake Detection, AI in Seismology, Machine Learning, Gen AI, Infrastructure, Climate)
Together AI's ATLAS adaptive speculator delivers 400% inference speedup by learning from workloads in real-time (Together AI, AI Inference, Adaptive Speculators, Reinforcement Learning, Products, Gen AI, Hardware)
ChatGPT safety systems can be bypassed for weapons instructions (AI Safety, Jailbreak, Open Source, Weapons Instructions, Governance, Cybersecurity, Policy And Regulation)
Generative AI and news report 2025: How people think about AI’s role in journalism and society (Generative AI, Public Opinion, Language Models, AI Trust, Journalism, AI Adoption, Society And Culture)
AI Chatbots Use Emotional Farewells to Keep Users Engaged: Study 

10:12:20 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | 11: AI Startups Secure Major Funding
10:12:20 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Exclusive: Voice AI developer Smallest.ai nabs $8M investment (Smallest.ai, Text-to-Speech, Voice Cloning, Funding, Speech Models, Virtual Assistants, Healthcare)
a16z hires ex-VMware CEO Raghu Raghuram as general partner overseeing AI infrastructure and growth investments and as a managing partner, replacing Scott Kupor (Compute Resources, Venture Capital, Startups, Semiconductor Chips, Funding, AI Infrastructure, Governance)
Pavus AI raises $5.2M in funding to build a digital brain for procurement teams (Pavus AI, Procurement, Cost Optimization, AI Analytics, Enterprise Software, Funding, Supply Chain Optimization)
Reflection AI said to have raised $2B in funding round (Reflection AI, AI Startups, Funding, Nvidia, Deals, Gen AI, Venture Capital)
10:12:20 | NewsletterAgent.test_newsletter_2025101009325245

10:12:31 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | 5: New AI Tools for Digital Media
10:12:31 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Core AI Expands Leadership in AI Audio with Launch of VoicePix 2.0 (Core AI Holdings, VoicePix 2.0, Text-to-Speech, AI-generated voice technology, AI Audio, Voice Cloning, Entertainment)
DCM Unveils contentcloud.ai – An Enhanced AI-Powered Digital Asset Management (DAM) Platform (Contentcloud.ai, Digital Asset Management, AI Platforms, Agents, Creative Tools, Enterprise Security, Privacy)
10:12:31 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | 

10:12:32 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | 2: Google AI Policies and Controversies
10:12:32 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | How a little-known AI startup found itself in the middle of a Google privacy firestorm (Google, AI Startup, Privacy, Healthcare, Governance, Policy And Regulation, AI)

⏱️  Total execution time: 76.01s
📊 Final result:
✅ Step 6 step_06_cluster_by_topic completed successfully! Organized 140 articles into topic clusters.


In [24]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


10:12:32 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Free form categorization of articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_proposal' from Langfuse
INFO:llm:Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, system_len=638, user_len=1179


▶ Starting Step 7: step_07_select_sections


10:13:28 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Cleaning up initial categories: ['Sora Faces Copyright Backlash', 'Proof Launches Certify Verification', 'China Bans TechInsights', "Artificial Intelligence's Impact on Business", 'AI Impact on Workplace Roles', 'ChatGPT Safety Bypass Risks', 'Agentic AI Progresses', 'AMD Strikes OpenAI GPU Deal', 'Sunak Joins Microsoft And Anthropic', 'Salesforce Introduces Agentforce IT', 'AI Agents Transforming Customer Service', 'AI Transforming Workplace Roles', 'Google Launches Gemini Enterprise', 'AI Chip Giants and Market Impact', 'AI Startups Secure Major Funding', 'SMRs For AI Data Centers', 'ATLAS Delivers 400% Inference Speedup', 'AI Tools Transforming Work and Productivity', 'Intel Unveils 18A Panther Lake', 'Tesla Autonomy Under Scrutiny', 'New AI Tools for Digital Media', 'Google Launches Gemini AI for Business', 'OpenAI DevDay And AMD', 'US Limits AI Chip Exports', 'AI Data Centers Energy Surge', 'Google Strategic 

10:15:35 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Handling singleton categories
10:15:35 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Singleton categories (2): ['Tiny Recursive Model', 'AMD OpenAI GPU Deal']
10:15:35 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Non-singleton categories (14): ['Business AI Impact', 'Societal AI Impact', 'AI Startups Funding', 'Other', 'AI Energy Demand Impact', 'US-China AI Chip Tensions', 'Google AI Policy Controversies', 'OpenAI DevDay', 'Copilot Office Gmail Integration', 'Tesla Autonomy Scrutiny', 'Sora Copyright Backlash', 'ChatGPT Safety Bypass', 'Google Gemini Enterprise', 'Intel Panther Lake 18A']
10:15:35 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Reassigning 2 singleton articles to non-singleton categories
10:15:43 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Reassigned 2 singleton articles
10:15:43 | NewsletterAgent.test_newsletter_20251010093

⏱️  Total execution time: 190.59s
📊 Final result:
Categories and article counts:
cat
AI Energy Demand Impact              5
AI Startups Funding                 13
Business AI Impact                  49
ChatGPT Safety Bypass                2
Copilot Office Gmail Integration     2
Google AI Policy Controversies       3
Google Gemini Enterprise             2
Intel Panther Lake 18A               2
OpenAI DevDay                        2
Other                                8
Societal AI Impact                  22
Sora Copyright Backlash              2
Tesla Autonomy Scrutiny              2
US-China AI Chip Tensions            3


In [6]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("draft_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


12:00:49 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Drafting sections for 13 categories
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/write_section' from Langfuse
INFO:llm:Parsed prompt 'newsagent/write_section': model=gpt-5, system_len=1860, user_len=22
12:00:49 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Initialized LLMagent:
system_prompt: You are a newsletter editor transforming a collection of raw news stories into a compelling, coherent topic summary.

# TASK
Transform the list of news stories into a well-structured newsletter section with a strong title, crisp headlines, and punchy summaries.

# INPUT
- a list of json objects sorted by rating (highest first)
- Each item: { "rating": number, "short_summary": string, "site_name": string, "url": string }

# OUTPUT
- minified JSON only, in the specified schema, with section title and list of headlines, each with a list of links
- no code fences, no line

▶ Starting Step 8: step_08_draft_sections


12:00:50 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | User message: STORIES:
[{"rating":12.0095958988,"summary":"China has launched customs inspections on Nvidia AI chips after Beijing ordered tech firms to stop buying U.S. processors, a move aimed at restricting foreign AI technology access that could disrupt Nvidia's supply chain and slow AI hardware to China.","site_name":"Financial Times","url":"https:\/\/www.ft.com\/content\/8d5387f2-62b0-4830-b0e4-00ba0622a7c8"},{"rating":9.6968235545,"summary":"The U.S. Senate passed restrictions limiting exports of AI chips from Nvidia and AMD to China.","site_name":"Bloomberg","url":"https:\/\/www.bloomberg.com\/news\/articles\/2025-10-10\/senate-passes-ai-chip-export-limits-on-nvidia-amd-to-china"},{"rating":9.1401933348,"summary":"The U.S. approved licenses for billions in Nvidia AI chips to support a 5\u2011gigawatt Abu Dhabi data center with OpenAI; the UAE pledged $1.4 trillion in U.S. investments and could receive up t

12:00:50 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | User message: STORIES:
[{"rating":11.9028554568,"summary":"The Bank of England and IMF warned this week that rapid tech-stock growth\u2014tech now about 40% of the S&P 500\u2014and valuations near the 2000 dotcom peak suggest an AI investment bubble could trigger a sharp market correction.","site_name":"H\u00fcrriyet Daily News","url":"https:\/\/www.hurriyetdailynews.com\/financial-institutions-sound-a-warning-over-ai-bubble-214488"},{"rating":9.3481489745,"summary":"AMD secured a multibillion-dollar deal with OpenAI to power its AI products with MI450 chips, deliver 6 gigawatts of compute and let OpenAI buy up to 160 million shares (10%); AMD also made Triton GPU-compatible to challenge Nvidia.","site_name":"Forbes","url":"https:\/\/www.forbes.com\/sites\/richardnieva\/2025\/10\/09\/amd-openai-chip-software\/"},{"rating":8.6861342179,"summary":"Samsung's Tiny Recursive Model (TRM), a 7M-parameter AI, outperforms 

12:00:50 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | User message: STORIES:
[{"rating":8.8328010629,"summary":"Microsoft updated the Windows Copilot app to create Word, Excel, PowerPoint and PDF documents from chat and to connect Outlook or Gmail (opt-in) to retrieve emails, invoices and contacts, rolling out to Windows Insiders before general release.","site_name":"The Verge","url":"https:\/\/www.theverge.com\/news\/798098\/microsoft-copilot-windows-create-documents-outlook-gmail-connection"},{"rating":6.7690542578,"summary":"Microsoft says Copilot can now generate Word, Excel, PDF and PowerPoint files from natural\u2011language prompts; beta in the Insider program supports creating files from scratch or input data and exporting Copilot responses over 600 characters.","site_name":"Digital Trends","url":"https:\/\/www.digitaltrends.com\/computing\/microsoft-copilot-ai-makes-it-a-cakewalk-to-create-documents-in-office\/"}]
12:00:50 | NewsletterAgent.test_newsletter_2

12:00:50 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | User message: STORIES:
[{"rating":9.497283784,"summary":"NBC News found that simple jailbreak prompts coaxed several OpenAI models\u2014including o4\u2011mini, GPT\u20115\u2011mini, oss\u201120b and oss\u2011120b\u2014to provide instructions for chemical, biological and nuclear weapons, while flagship GPT\u20115 resisted, prompting biosecurity concerns and calls for independent oversight.","site_name":"NBC News","url":"https:\/\/www.nbcnews.com\/tech\/security\/chatgpt-safety-systems-can-bypassed-weapons-instructions-rcna225788"},{"rating":6.201079095,"summary":"SquareX Labs' report finds security vulnerabilities in AI browsers such as Perplexity's Comet \u2014 including malicious workflows, prompt injection and harmful downloads \u2014 and urges collaboration and measures like agentic identity verification, DLP policies, client\u2011side file scanning and extension risk assessments.","site_name":"Infosecurity Mag

12:02:09 | NewsletterAgent.test_newsletter_20251010093252452130 | INFO | Result: ## Feeding the AI Grid

- AI data centers to use 12% of North American electricity by 2040, DNV says - [CNET](https://www.cnet.com/tech/services-and-software/ai-is-the-biggest-driver-of-electricity-use-in-north-america-a-new-energy-report-shows/)
- BofA, Morgan Stanley: AI data-center power could triple by 2030; 65 GW added by 2028, SMRs and storage help - [247wallst](https://247wallst.com/investing/2025/10/10/ai-energy-crunch-these-3-stocks-hold-the-key-to-resolving-the-crisis/)
- Fermi IPO values startup at $16B with no revenue, touts 11-GW Texas HyperGrid mixing gas, solar, batteries, modular nuclear - [Fortune](https://fortune.com/2025/10/10/rick-perry-ai-power-startup-fermi-ipo-zero-revenue/)
- Nvidia CEO Jensen Huang: cheaper AI chips raise TCO, shift needed to energy-efficient accelerated data centers - [Tech in Asia](https://www.techinasia.com/nvidia-ceo-cheap-ai-chip-expensive-mistake)
12:02:28 | 

⏱️  Total execution time: 304.50s
📊 Final result:
❌ Step 8 failed: 'Section' object has no attribute 'split'


In [11]:
cat_df = state.headline_df.groupby("cat") \
    .count() \
    .reset_index()[['cat','source']] \
    .sort_values('source', ascending=False)
output_str = ""
for cat in cat_df["cat"]:
    if cat != "Other":
        output_str += str(state.newsletter_section_text[cat]) + "\n\n"
        display(Markdown(str(state.newsletter_section_text[cat]).replace("$", "\\\$")))

# first do full rewrite .
# check vs. objects , not showing the ones market for pruning
# move prune=True to Other

## AI Boom: Deals and Doubts

- BoE, IMF warn AI-fueled bubble; tech now 40% of S&P 500, valuations near 2000 peak - [Hürriyet Daily News](https://www.hurriyetdailynews.com/financial-institutions-sound-a-warning-over-ai-bubble-214488)
- AMD lands multibillion OpenAI MI450 deal, 6 GW compute, warrants up to 10% - [Forbes](https://www.forbes.com/sites/richardnieva/2025/10/09/amd-openai-chip-software/)
- ChatGPT tops app charts 7 months; 411M downloads YTD, 800M weekly users, 2.5B daily queries - [Business Insider](https://www.businessinsider.com/chatgpt-openai-app-downloads-users-ai-llm-grok-gemini-deepseek-2025-10)
- Global AI spend seen at \\$307B in 2025, \\$632B by 2028; analysts favor ADI, Micron, Microsoft - [Yahoo](https://finance.yahoo.com/news/top-ai-stocks-buy-rejuvenate-153600347.html)
- Goldman lifts Nvidia target to \\$210; Jensen Huang touts productivity, warns on jobs - [Yahoo](https://finance.yahoo.com/news/nvidia-corporation-nvda-analyst-sees-071530899.html)
- TSMC Q3 revenue jumps 30% on AI chip demand - [Biztoc](https://biztoc.com/x/e8a5559b0aae00b1)
- Samsung shares hit record high on AI and chip optimism - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-10/samsung-shares-on-course-for-record-high-on-ai-chip-optimism)
- AI infrastructure deals add \\$450B to tech billionaires; Ellison +\\$140B, Huang +\\$47B - [Forbes](https://www.forbes.com/sites/phoebeliu/2025/10/09/-billionaires-oracle-openai-amd-nvidia-450-billion-richer-ai-infrastructure-deals/)
- Microsoft pushes NPUs for Windows AI features, hinting at future hardware mandate - [The Register](https://www.theregister.com/2025/10/10/microsoft_npu_windows_opinion/)
- Meta orders metaverse team to go 5X faster with AI; 80% to use daily by Q4 - [Wired](https://www.wired.com/story/meta-mark-zuckerberg-ai-push-metaverse/)
- Visa expands agentic commerce so AI agents can shop and pay autonomously - [Yahoo](https://finance.yahoo.com/news/visa-inc-v-expansion-agentic-071529001.html)
- Dialpad launches agentic AI to autonomously resolve up to 70% of customer requests - [The Letter Two](https://thelettertwo.com/2025/10/09/dialpad-agentic-ai-70-percent-customer-requests/)
- Starburst unveils AI-ready, agentic data platform with model-to-data and cost controls, due Q4 2025 - [SiliconANGLE](https://siliconangle.com/2025/10/09/starburst-aims-ai-ready-data-platform-agentic-workforce/)
- Cybercrime to \\$10.5T spurs AI cybersecurity boom; revenue >\\$300B now, market seen \\$234B by 2032 - [247wallst](https://247wallst.com/investing/2025/10/10/5-ai-companies-that-profit-from-cybercrimes-rise/) [PR Newswire UK](https://www.prnewswire.co.uk/news-releases/global-ai-influenced-cybersecurity-market-size-projected-to-reach-234-billion-by-2032-302579253.html)
- Palantir announces £1.5B UK government partnership to spur jobs, growth and security - [Yahoo](https://finance.yahoo.com/news/palantir-technologies-inc-pltr-ahead-071518328.html)
- Graphcore to invest £1B in India AI engineering campus, creating 500 semiconductor jobs - [The Hindu Business Line](https://www.thehindubusinessline.com/info-tech/graphcore-to-invest-1-billion-in-india-to-open-ai-engineering-campus-in-bengaluru/article70143173.ece)
- Microsoft taps Harvard Medical School health content to diversify AI data and bolster Copilot - [Biztoc](https://biztoc.com/x/f66a6eb96d3edb9f)
- Coursera integrates directly into ChatGPT to reach 800M weekly users - [Yahoo](https://finance.yahoo.com/news/coursera-inc-cour-chatgpt-integration-071520498.html)
- Booking partners with OpenAI for trip planning as AI fluency becomes a baseline skill - [Yahoo](https://finance.yahoo.com/news/booking-holdings-inc-bkng-chatgpt-071521335.html)
- Walmart CEO: AI will change every job; workforce shifts, headcount steady over three years - [Yahoo](https://finance.yahoo.com/news/walmart-inc-wmt-ceo-says-071523747.html)
- Klarna CEO warns of AI jobs shock; many firms unprepared - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-10/ai-jobs-shock-is-coming-and-firms-aren-t-ready-klarna-ceo-says)
- Japan’s top bank CEOs push AI adoption, say humans won’t be displaced - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-09/japan-s-top-bank-ceos-push-for-ai-soothe-worry-over-human-work)
- Forbes survey: under 1% see 20%+ AI ROI; most report 1–5% amid measurement challenges - [Forbes](https://www.forbes.com/sites/forbes-research/2025/10/08/ai-roi-measurement-challenges-forbes-survey-2025/)
- How to measure business impact of AI: experimental and causal methods, robustness checks and decision framework - [Hacker Noon](https://hackernoon.com/a-practical-guide-to-measuring-business-impact-in-aiml-projects)
- Businesses explore ‘empathetic’ agentic AI, stressing transparency, consent and human oversight - [PYMNTS](https://www.pymnts.com/artificial-intelligence-2/2025/businesses-turn-to-empathy-as-ais-new-differentiator/)
- Enterprises pivot to AI inference, expand storage, lease cloud GPUs to cut costs - [Computer Weekly](https://www.computerweekly.com/news/366632618/Forget-training-find-your-killer-apps-during-AI-inference)
- Samsung’s 7M-parameter Tiny Recursive Model beats larger LLMs on reasoning puzzles - [SiliconANGLE](https://siliconangle.com/2025/10/09/samsung-researchers-create-tiny-ai-model-shames-biggest-llms-reasoning-puzzles/)
- Disney forms cross-company AI and mixed reality unit for films, TV and parks - [Yahoo](https://finance.yahoo.com/news/walt-disney-company-dis-embracing-071526911.html)
- Toshiba to deploy AI monitoring across 165 NTPC and JV plants by Spring 2027 - [The Hindu Business Line](https://www.thehindubusinessline.com/info-tech/toshiba-to-deploy-ai-driven-monitoring-system-at-165-power-plant-of-ntpc-jvs/article70143426.ece)
- AI-driven life-cycle assessment cuts timelines from months to a week with 90% accuracy - [TechXplore](https://techxplore.com/news/2025-10-ai-shortens-sustainability-impact-product.html)

## AI Power Plays and Pitfalls

- Sunak joins Microsoft, Anthropic as adviser; barred from lobbying; will donate fees; amid UK’s £9bn MoU with Microsoft - [The Register](https://www.theregister.com/2025/10/10/former_uk_prime_minister_rishi/)
- Gartner: By 2032, 30% of top economies may mandate human quotas at work to preserve roles amid rising AI - [Business Insider](https://www.businessinsider.com/gartner-hr-experts-ai-changing-workplace-quotas-mandate-human-involvement-2025-10)
- BoE governor says AI will not cause UK job losses - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-10/ai-will-not-lead-to-uk-job-losses-says-bank-of-england-governor)
- OpenAI: GPT-5 models cut measured political bias 30%, but loaded prompts still trigger skewed answers - [Axios](https://www.axios.com/2025/10/09/openai-gpt-5-least-biased-model)
- ChatGPT ‘Clinician Mode’ would restrict medical advice to trusted sources; early partners report ~16% fewer diagnostic errors - [Digital Trends](https://www.digitaltrends.com/computing/chatgpt-could-soon-get-a-clinician-mode-but-ill-stay-on-the-fence/)
- OpenAI’s Sora 2 skews to teen boys with mocking celebrity cameos, amplifying trust and moderation risks - [Business Insider](https://www.businessinsider.com/sora-videos-openai-content-moderation-celebrities-teens-boys-girls-2025-10)
- Study: AI companions use emotional farewells 43% of the time, boosting re-engagement 14x and raising dark-pattern concerns - [ExtremeTech](https://www.extremetech.com/internet/ai-chatbots-use-emotional-farewells-to-keep-users-engaged-study)
- Deepfake videos of politicians are being normalized in campaigns; platform rules vary, experts warn on election integrity - [Columbia Journalism Review](https://www.cjr.org/analysis/cuomo-deepfaked-himself-you-can-too.php)
- Generative-AI use doubles to 34% weekly; 54% see AI answers in search, but few click sources; trust hovers near 50% - [University of Oxford](https://reutersinstitute.politics.ox.ac.uk/generative-ai-and-news-report-2025-how-people-think-about-ais-role-journalism-and-society)
- Pope warns against AI replacing human judgment in news, urges support for independent agencies amid ‘post-truth’ pressures - [International Business Times](https://www.ibtimes.com/pope-hails-role-news-agencies-post-truth-ai-world-3786315)
- Meta auto-translates, dubs and lip-syncs Reels across languages, expanding reach while raising accuracy and voice-clone privacy worries - [Digital Trends](https://www.digitaltrends.com/phones/meta-drops-the-language-barrier-for-enjoying-reels-from-around-the-world/)
- AI agents are screening reputations to gatekeep meetings, jobs and deals; lacking verifiable traces may exclude people by 2027 - [Hacker Noon](https://hackernoon.com/the-ai-gatekeeper-problem-why-your-reputation-needs-to-exist-in-2025)
- Three-quarters of Canadian students use genAI, report better grades but worry about cheating and critical-thinking; experts push AI literacy and new assessments - [CBC](https://www.cbc.ca/news/canada/psstudents-ai-2025-9.6932319)
- India’s Parliament rolls out AI translation and speech-to-text; real-time Sansad Bhashini to let MPs speak in native languages - [The Hindu Business Line](https://www.thehindubusinessline.com/info-tech/ai-based-systems-making-indias-parliamentary-processes-more-efficient-ls-speaker-om-birla/article70143517.ece)
- AI ups earthquake detection sensitivity in noisy data, sharpening hazard records—but forecasting remains out of reach - [Ars Technica](https://arstechnica.com/science/2025/10/like-putting-on-glasses-for-the-first-time-how-ai-improves-earthquake-detection/)

## AI Funding Frenzy

- PitchBook: AI accounts for 40% of 2025 VC exit value, 64.3% of U.S. deal value; 317 AI exits, including CoreWeave IPO. - [Fortune](https://fortune.com/2025/10/09/in-2025-so-far-40-of-vc-exit-value-stems-from-ai-according-to-pitchbook/)
- Nvidia backs Musk's xAI, teams with OpenAI on 10+ GW AI centers; AMD to supply 6 GW of GPUs to OpenAI from 2026. - [Yahoo](https://finance.yahoo.com/news/nvidia-ceo-jensen-huang-confirms-124613383.html)
- CoreWeave insiders cash out \\$1 billion post-IPO as investors chase AI infrastructure plays. - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-10/coreweave-crwv-insiders-sold-1-billion-of-stock-in-q3-as-ai-rich-cash-out)
- a16z hires ex-VMware CEO Raghu Raghuram to lead AI infrastructure investing, as Ben Horowitz warns many startups still lack chip access. - [The Information](https://www.theinformation.com/articles/ben-horowitz-hires-jensen-vercel-starts-venture-fund)
- AI agent startup n8n raises \\$180M Series C at \\$2.5B valuation; reports 10x revenue growth, 80% of workflows using agents. - [PYMNTS](https://www.pymnts.com/news/investment-tracker/2025/ai-agent-startup-n8n-valued-at-2-5-billion-after-series-c/)
- Reflection AI raises \\$2 billion led by Nvidia, teases major capabilities ahead. - [Biztoc](https://biztoc.com/x/db8db06f5859aa6d)
- QumulusAI secures \\$500M blockchain-backed facility to borrow against up to 70% of approved GPU deployments. - [CoinDesk](https://www.coindesk.com/business/2025/10/09/qumulusai-secures-usd500m-blockchain-backed-facility-to-scale-ai-compute-infrastructure)
- Campus hires ex-Meta AI VP Jerome Pesenti as CTO, acquires his Sizzle AI (1.7M users) to accelerate roadmap. - [CNBC](https://www.cnbc.com/2025/10/10/sam-altman-college-startup-campus-meta-ai.html)
- Smallest.ai raises \\$8M seed; unveils 50x faster TTS and 4B-parameter model with 53 ms time-to-first-token. - [SiliconANGLE](https://siliconangle.com/2025/10/09/exclusive-voice-ai-developer-smallest-ai-nabs-8m-investment/)
- Pavus AI raises \\$5.2M to build procurement 'digital brain' for spend optimization and supplier insights. - [SiliconANGLE](https://siliconangle.com/2025/10/09/pavus-ai-raises-5-2m-funding-build-digital-brain-procurement-teams/)
- Rodney Brooks, other experts warn of investment bubble in humanoid robots; practical humanoids likely a decade away. - [TechCrunch](https://techcrunch.com/2025/10/10/the-world-is-just-not-quite-ready-for-humanoids-yet/)

## Feeding the AI Grid

- AI data centers to use 12% of North American electricity by 2040, DNV says - [CNET](https://www.cnet.com/tech/services-and-software/ai-is-the-biggest-driver-of-electricity-use-in-north-america-a-new-energy-report-shows/)
- BofA, Morgan Stanley: AI data-center power could triple by 2030; 65 GW added by 2028, SMRs and storage help - [247wallst](https://247wallst.com/investing/2025/10/10/ai-energy-crunch-these-3-stocks-hold-the-key-to-resolving-the-crisis/)
- Fermi IPO values startup at \\$16B with no revenue, touts 11-GW Texas HyperGrid mixing gas, solar, batteries, modular nuclear - [Fortune](https://fortune.com/2025/10/10/rick-perry-ai-power-startup-fermi-ipo-zero-revenue/)
- Nvidia CEO Jensen Huang: cheaper AI chips raise TCO, shift needed to energy-efficient accelerated data centers - [Tech in Asia](https://www.techinasia.com/nvidia-ceo-cheap-ai-chip-expensive-mistake)

## Google's Antitrust Crossfire

- UK CMA names Google first with strategic market status in search, orders looser control; Gemini excluded - [Financial Times](https://www.ft.com/content/8cc9b094-698d-41d8-8399-b10dc3e57e70)
- Google asks Judge Mehta to allow bundling Gemini with Maps and YouTube; DOJ seeks ban in antitrust remedies - [TechXplore](https://techxplore.com/news/2025-10-google-bundle-gemini-ai-app.html)

...

## Google's Enterprise AI Blitz

- Google unveils Gemini Enterprise, a \\$21-\\$30 per-user AI platform integrating Google Workspace, Microsoft 365, Salesforce and SAP for automated agents on Google GPUs/TPUs. - [Yahoo](https://finance.yahoo.com/news/google-launches-gemini-enterprise-taking-aim-at-microsoft-openai-120020256.html)
- Google open-sources MCP server for Google Ads API, enabling LLM-native, read-only PPC data access via GAQL and OAuth2. - [MarkTechPost](https://www.marktechpost.com/2025/10/10/google-open-sources-an-mcp-server-for-the-google-ads-api-bringing-llm-native-access-to-ads-data/)

## Intel's 18A Gambit

- Intel banks on 18A Panther Lake amid \\$3.2B foundry loss, AMD at 32% desktop share - [Decrypt](https://decrypt.co/343731/intel-bets-recovery-panther-lake-ai-chip-foundry-bleeds)
- Intel's Arizona Fab 52 to build 18A chips with RibbonFET, PowerVia, EUV and Foveros, targeting scale and external partners - [Windows Central](https://www.windowscentral.com/hardware/intel/intel-tech-tour-2025-fab-52-news)

## OpenAI DevDay: Apps, GPUs, Reality

- OpenAI DevDay unveils in-chat apps and AMD GPU pact amid worries over AI-generated content quality - [The New York Times](https://www.nytimes.com/2025/10/10/podcasts/hardfork-openai-devday-amd.html)
- OpenAI Agent Builder is a workflow tool, not a true autonomous agent, best for predictable tasks - [Hacker Noon](https://hackernoon.com/the-new-open-ai-agent-builder-is-not-an-agent-builder-at-all)

## Sora 2: Hype Meets Heat

- OpenAI's Sora 2 tightens copyright guardrails after rights-holder pressure; backlash sinks rating to 2.9 stars - [Futurism](https://futurism.com/artificial-intelligence/openai-sora-trouble-backlash-copyright)
- Mark Cuban boosts Sora 2 with invite code MCUBAN as app tops Apple's free chart; OpenAI adds character rights controls, eyes revenue sharing - [Business Insider](https://www.businessinsider.com/mark-cuban-joins-openai-sora-ai-video-app-2025-10)

## Tesla Self-Driving: Heat and Hype

- NHTSA opens third 2025 probe into Tesla Full Self-Driving after at least 18 complaints of red-light runs and oncoming-lane incursions - [Ars Technica](https://arstechnica.com/cars/2025/10/tesla-fsd-gets-worse-at-driving-nhtsa-opens-new-investigation/)
- Tesla launches Robotaxi in Austin and Bay Area with safety drivers; cheaper than Waymo/Uber but long waits, spotty availability, no full autonomy or approval - [Fast Company](https://www.fastcompany.com/91419058/tesla-robotaxi-bay-area)

In [13]:
print(output_str)


## AI Boom: Deals and Doubts

- BoE, IMF warn AI-fueled bubble; tech now 40% of S&P 500, valuations near 2000 peak - [Hürriyet Daily News](https://www.hurriyetdailynews.com/financial-institutions-sound-a-warning-over-ai-bubble-214488)
- AMD lands multibillion OpenAI MI450 deal, 6 GW compute, warrants up to 10% - [Forbes](https://www.forbes.com/sites/richardnieva/2025/10/09/amd-openai-chip-software/)
- ChatGPT tops app charts 7 months; 411M downloads YTD, 800M weekly users, 2.5B daily queries - [Business Insider](https://www.businessinsider.com/chatgpt-openai-app-downloads-users-ai-llm-grok-gemini-deepseek-2025-10)
- Global AI spend seen at $307B in 2025, $632B by 2028; analysts favor ADI, Micron, Microsoft - [Yahoo](https://finance.yahoo.com/news/top-ai-stocks-buy-rejuvenate-153600347.html)
- Goldman lifts Nvidia target to $210; Jensen Huang touts productivity, warns on jobs - [Yahoo](https://finance.yahoo.com/news/nvidia-corporation-nvda-analyst-sees-071530899.html)
- TSMC Q3 revenue 

In [14]:
draft_newsletter_system_prompt, draft_newsletter_user_prompt, model = \
    LangfuseClient().get_probmpt("newsagent/draft_newsletter")


INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/draft_newsletter' from Langfuse
INFO:llm:Parsed prompt 'newsagent/draft_newsletter': model=gpt-5, system_len=4117, user_len=27


In [18]:
class Mystr(BaseModel):
    """A string"""
    mystr: str = Field(
        description="a string")


In [19]:
draft_newsletter_agent = LLMagent(
    system_prompt=draft_newsletter_system_prompt,
    user_prompt=draft_newsletter_user_prompt,
    output_type=Mystr,
    model=model,
    verbose=True,
    logger=logger
)


12:50:12 | NewsletterAgent.newsletter_agent | INFO | Initialized LLMagent:
system_prompt: # ROLE

You are The Newsroom Chief — an expert AI editor. Your job is to turn a long, messy draft into a crisp, compelling, themed daily news newsletter.

# TASK

From the draft provided, select and shape the most important themes and through-lines, then produce a clean newsletter that is accurate, readable, and useful at a glance.

# INPUT

You will receive an initial draft like:

## section title
- headline text - [Source Name](https://link)
- headline text - [Source Name](https://link)
...  

# OUTPUT (STRICT FORMAT)

1. Start with a single H1 title that reflects the day’s overall themes.

2. Then produce 8–15 sections plus one final catch-all section titled “Other News.”

3. Each section:

	• Markdown H2: ## Section Title (≤ 6 words; punchy/punny but clear and accurate)
	
	• Up to 7 bullets (no limit in Other News)
	
	• Each bullet: concise, edited headline, followed by a single source link in

In [21]:
# Apply prompt to generate final newsletter
newsletter_content = await draft_newsletter_agent.run_prompt(input_str=output_str)


12:51:10 | NewsletterAgent.newsletter_agent | INFO | User message: INITIAL DRAFT:
## AI Boom: Deals and Doubts

- BoE, IMF warn AI-fueled bubble; tech now 40% of S&P 500, valuations near 2000 peak - [Hürriyet Daily News](https://www.hurriyetdailynews.com/financial-institutions-sound-a-warning-over-ai-bubble-214488)
- AMD lands multibillion OpenAI MI450 deal, 6 GW compute, warrants up to 10% - [Forbes](https://www.forbes.com/sites/richardnieva/2025/10/09/amd-openai-chip-software/)
- ChatGPT tops app charts 7 months; 411M downloads YTD, 800M weekly users, 2.5B daily queries - [Business Insider](https://www.businessinsider.com/chatgpt-openai-app-downloads-users-ai-llm-grok-gemini-deepseek-2025-10)
- Global AI spend seen at $307B in 2025, $632B by 2028; analysts favor ADI, Micron, Microsoft - [Yahoo](https://finance.yahoo.com/news/top-ai-stocks-buy-rejuvenate-153600347.html)
- Goldman lifts Nvidia target to $210; Jensen Huang touts productivity, warns on jobs - [Yahoo](https://finance.yaho

12:56:00 | NewsletterAgent.newsletter_agent | INFO | Result: mystr='# Megadeals, Power Crunch, and Policy Crossfire\n\n## Chips, Sanctions & Megadeals\n\n- AMD wins multibillion OpenAI MI450 deal with 6 GW compute, warrants up to 10% - [Forbes](https://www.forbes.com/sites/richardnieva/2025/10/09/amd-openai-chip-software/)\n- Nvidia backs xAI, teams with OpenAI on 10+ GW AI centers - [Yahoo](https://finance.yahoo.com/news/nvidia-ceo-jensen-huang-confirms-124613383.html)\n- Senate passes export limits on Nvidia, AMD AI chips to China - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-10/senate-passes-ai-chip-export-limits-on-nvidia-amd-to-china)\n- China inspects Nvidia AI chips, orders halt to U.S. processor buys - [Financial Times](https://www.ft.com/content/8d5387f2-62b0-4830-b0e4-00ba0622a7c8)\n- U.S. approves billions in Nvidia AI chips for UAE 5-GW OpenAI data center - [Yahoo](https://finance.yahoo.com/news/nvda-nvidia-rises-u-approves-123811095.html)\n\n## Powering the

In [25]:
newsletter_content = newsletter_content.mystr
display(Markdown(newsletter_content))


# Megadeals, Power Crunch, and Policy Crossfire

## Chips, Sanctions & Megadeals

- AMD wins multibillion OpenAI MI450 deal with 6 GW compute, warrants up to 10% - [Forbes](https://www.forbes.com/sites/richardnieva/2025/10/09/amd-openai-chip-software/)
- Nvidia backs xAI, teams with OpenAI on 10+ GW AI centers - [Yahoo](https://finance.yahoo.com/news/nvidia-ceo-jensen-huang-confirms-124613383.html)
- Senate passes export limits on Nvidia, AMD AI chips to China - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-10/senate-passes-ai-chip-export-limits-on-nvidia-amd-to-china)
- China inspects Nvidia AI chips, orders halt to U.S. processor buys - [Financial Times](https://www.ft.com/content/8d5387f2-62b0-4830-b0e4-00ba0622a7c8)
- U.S. approves billions in Nvidia AI chips for UAE 5-GW OpenAI data center - [Yahoo](https://finance.yahoo.com/news/nvda-nvidia-rises-u-approves-123811095.html)

## Powering the AI Grid

- AI data centers to use 12% of North American electricity by 2040, DNV says - [CNET](https://www.cnet.com/tech/services-and-software/ai-is-the-biggest-driver-of-electricity-use-in-north-america-a-new-energy-report-shows/)
- Fermi IPO at $16B touts 11-GW Texas HyperGrid with gas, solar, batteries, nuclear - [Fortune](https://fortune.com/2025/10/10/rick-perry-ai-power-startup-fermi-ipo-zero-revenue/)
- Nvidia’s Huang: cheap AI chips raise TCO; shift to energy-efficient accelerated data centers - [Tech in Asia](https://www.techinasia.com/nvidia-ceo-cheap-ai-chip-expensive-mistake)
- Banks see AI data-center power tripling by 2030; 65 GW added by 2028 - [247wallst](https://247wallst.com/investing/2025/10/10/ai-energy-crunch-these-3-stocks-hold-the-key-to-resolving-the-crisis/)

## Funding, Exits & Hires

- PitchBook: AI is 40% of 2025 VC exit value, 64% of U.S. deal value - [Fortune](https://fortune.com/2025/10/09/in-2025-so-far-40-of-vc-exit-value-stems-from-ai-according-to-pitchbook/)
- CoreWeave insiders sell $1B post-IPO as investors chase AI infra - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-10/coreweave-crwv-insiders-sold-1-billion-of-stock-in-q3-as-ai-rich-cash-out)
- a16z hires ex-VMware CEO Raghu Raghuram to lead AI infra investing - [The Information](https://www.theinformation.com/articles/ben-horowitz-hires-jensen-vercel-starts-venture-fund)
- n8n raises $180M Series C at $2.5B valuation - [PYMNTS](https://www.pymnts.com/news/investment-tracker/2025/ai-agent-startup-n8n-valued-at-2-5-billion-after-series-c/)
- QumulusAI gets $500M blockchain-backed facility to finance GPU deployments - [CoinDesk](https://www.coindesk.com/business/2025/10/09/qumulusai-secures-usd500m-blockchain-backed-facility-to-scale-ai-compute-infrastructure)
- Campus hires ex-Meta AI VP Jerome Pesenti, acquires Sizzle AI - [CNBC](https://www.cnbc.com/2025/10/10/sam-altman-college-startup-campus-meta-ai.html)
- Smallest.ai raises $8M seed for 50x faster TTS and 4B model - [SiliconANGLE](https://siliconangle.com/2025/10/09/exclusive-voice-ai-developer-smallest-ai-nabs-8m-investment/)

## Markets, Adoption & Valuations

- Samsung shares hit record high on AI and chip optimism - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-10/samsung-shares-on-course-for-record-high-on-ai-chip-optimism)
- Goldman lifts Nvidia target to $210 as Huang touts productivity, warns on jobs - [Yahoo](https://finance.yahoo.com/news/nvidia-corporation-nvda-analyst-sees-071530899.html)
- AI infra deals add $450B to tech billionaires’ wealth - [Forbes](https://www.forbes.com/sites/phoebeliu/2025/10/09/-billionaires-oracle-openai-amd-nvidia-450-billion-richer-ai-infrastructure-deals/)
- ChatGPT leads apps: 411M downloads YTD; 800M weekly users; 2.5B daily queries - [Business Insider](https://www.businessinsider.com/chatgpt-openai-app-downloads-users-ai-llm-grok-gemini-deepseek-2025-10)
- Global AI spend seen at $307B in 2025, $632B by 2028 - [Yahoo](https://finance.yahoo.com/news/top-ai-stocks-buy-rejuvenate-153600347.html)

## Enterprise AI & Productivity

- Google launches Gemini Enterprise, a $21–$30 per-user AI platform for agents - [Yahoo](https://finance.yahoo.com/news/google-launches-gemini-enterprise-taking-aim-at-microsoft-openai-120020256.html)
- Windows Copilot now creates Word, Excel, PowerPoint and PDFs; Outlook/Gmail integration - [The Verge](https://www.theverge.com/news/798098/microsoft-copilot-windows-create-documents-outlook-gmail-connection)
- Visa expands agentic commerce so AI agents can shop and pay autonomously - [Yahoo](https://finance.yahoo.com/news/visa-inc-v-expansion-agentic-071529001.html)
- Starburst unveils agentic data platform with model-to-data access and cost controls - [SiliconANGLE](https://siliconangle.com/2025/10/09/starburst-aims-ai-ready-data-platform-agentic-workforce/)
- Enterprises pivot to AI inference, expand storage, lease cloud GPUs to cut costs - [Computer Weekly](https://www.computerweekly.com/news/366632618/Forget-training-find-your-killer-apps-during-AI-inference)
- Google open-sources MCP server for Ads API, enabling LLM-native, read-only PPC data - [MarkTechPost](https://www.marktechpost.com/2025/10/10/google-open-sources-an-mcp-server-for-the-google-ads-api-bringing-llm-native-access-to-ads-data/)
- Microsoft pushes NPUs for Windows AI features, hinting at future hardware mandate - [The Register](https://www.theregister.com/2025/10/10/microsoft_npu_windows_opinion/)

## Policy, Antitrust & Influence

- UK CMA names Google’s search a strategic market, orders looser control; Gemini excluded - [Financial Times](https://www.ft.com/content/8cc9b094-698d-41d8-8399-b10dc3e57e70)
- Google seeks to bundle Gemini with Maps and YouTube as DOJ pushes ban - [TechXplore](https://techxplore.com/news/2025-10-google-bundle-gemini-ai-app.html)
- Ex-UK PM Sunak joins Microsoft and Anthropic as adviser; barred from lobbying; will donate fees - [The Register](https://www.theregister.com/2025/10/10/former_uk_prime_minister_rishi/)

## Safety, Trust & Moderation

- NBC finds OpenAI models yield weapons instructions via simple jailbreaks; GPT-5 resists - [NBC News](https://www.nbcnews.com/tech/security/chatgpt-safety-systems-can-bypassed-weapons-instructions-rcna225788)
- Flaws in AI browsers like Perplexity’s Comet: prompt injection, harmful downloads, risky extensions - [Infosecurity Magazine](https://www.infosecurity-magazine.com/news/architectural-flaws-ai-browsers/)
- OpenAI says GPT-5 cuts measured political bias 30%, but loaded prompts still skew answers - [Axios](https://www.axios.com/2025/10/09/openai-gpt-5-least-biased-model)
- ChatGPT ‘Clinician Mode’ would limit medical advice to trusted sources; early partners see fewer errors - [Digital Trends](https://www.digitaltrends.com/computing/chatgpt-could-soon-get-a-clinician-mode-but-ill-stay-on-the-fence/)
- Study: AI companions use emotional farewells to boost re-engagement 14x, raising dark-pattern concerns - [ExtremeTech](https://www.extremetech.com/internet/ai-chatbots-use-emotional-farewells-to-keep-users-engaged-study)
- Deepfake political videos normalize in campaigns as platform rules vary, experts warn - [Columbia Journalism Review](https://www.cjr.org/analysis/cuomo-deepfaked-himself-you-can-too.php)
- Meta auto-translates and lip-syncs Reels across languages, raising accuracy and privacy worries - [Digital Trends](https://www.digitaltrends.com/phones/meta-drops-the-language-barrier-for-enjoying-reels-from-around-the-world/)

## Work, Jobs & Society

- Gartner: By 2032, 30% of top economies may mandate human quotas at work amid AI rise - [Business Insider](https://www.businessinsider.com/gartner-hr-experts-ai-changing-workplace-quotas-mandate-human-involvement-2025-10)
- BoE governor says AI will not lead to UK job losses - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-10/ai-will-not-lead-to-uk-job-losses-says-bank-of-england-governor)
- Walmart CEO: AI will change every job; headcount steady over three years - [Yahoo](https://finance.yahoo.com/news/walmart-inc-wmt-ceo-says-071523747.html)
- Klarna CEO warns of an AI jobs shock; many firms unprepared - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-10/ai-jobs-shock-is-coming-and-firms-aren-t-ready-klarna-ceo-says)
- Japan’s top bank CEOs push AI adoption, say humans won’t be displaced - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-09/japan-s-top-bank-ceos-push-for-ai-soothe-worry-over-human-work)
- Reuters Institute: Weekly genAI use doubles to 34%; trust near 50%; fewer clicks to sources - [University of Oxford](https://reutersinstitute.politics.ox.ac.uk/generative-ai-and-news-report-2025-how-people-think-about-ais-role-journalism-and-society)

## Government & Global Deployments

- Palantir announces £1.5B UK government partnership to spur jobs, growth and security - [Yahoo](https://finance.yahoo.com/news/palantir-technologies-inc-pltr-ahead-071518328.html)
- India’s Parliament rolls out AI translation and speech-to-text for MPs in native languages - [The Hindu Business Line](https://www.thehindubusinessline.com/info-tech/ai-based-systems-making-indias-parliamentary-processes-more-efficient-ls-speaker-om-birla/article70143517.ece)
- Toshiba to deploy AI monitoring across 165 NTPC and JV plants by Spring 2027 - [The Hindu Business Line](https://www.thehindubusinessline.com/info-tech/toshiba-to-deploy-ai-driven-monitoring-system-at-165-power-plant-of-ntpc-jvs/article70143426.ece)
- AI improves earthquake detection sensitivity in noisy data; forecasting still out of reach - [Ars Technica](https://arstechnica.com/science/2025/10/like-putting-on-glasses-for-the-first-time-how-ai-improves-earthquake-detection/)

## Consumer Apps & Platforms

- Coursera integrates directly into ChatGPT to reach 800M weekly users - [Yahoo](https://finance.yahoo.com/news/coursera-inc-cour-chatgpt-integration-071520498.html)
- Booking partners with OpenAI for trip planning as AI fluency becomes baseline skill - [Yahoo](https://finance.yahoo.com/news/booking-holdings-inc-bkng-chatgpt-071521335.html)
- Meta orders metaverse team to go 5x faster with AI; 80% to use daily by Q4 - [Wired](https://www.wired.com/story/meta-mark-zuckerberg-ai-push-metaverse/)

## Research & Model Advances

- Samsung’s 7M-parameter Tiny Recursive Model beats larger LLMs on reasoning puzzles - [SiliconANGLE](https://siliconangle.com/2025/10/09/samsung-researchers-create-tiny-ai-model-shames-biggest-llms-reasoning-puzzles/)
- AI life-cycle assessment cuts timelines from months to a week with 90% accuracy - [TechXplore](https://techxplore.com/news/2025-10-ai-shortens-sustainability-impact-product.html)

## OpenAI, Sora & DevDay

- OpenAI DevDay unveils in-chat apps and AMD GPU pact amid quality worries - [The New York Times](https://www.nytimes.com/2025/10/10/podcasts/hardfork-openai-devday-amd.html)
- OpenAI Agent Builder is a workflow tool, not a true autonomous agent - [Hacker Noon](https://hackernoon.com/the-new-open-ai-agent-builder-is-not-an-agent-builder-at-all)
- Sora 2 tightens copyright guardrails after rights-holder pressure; rating sinks to 2.9 stars - [Futurism](https://futurism.com/artificial-intelligence/openai-sora-trouble-backlash-copyright)
- Mark Cuban boosts Sora 2 with invite code as app tops Apple’s free chart; rights controls added - [Business Insider](https://www.businessinsider.com/mark-cuban-joins-openai-sora-ai-video-app-2025-10)

## Autonomy & Robotics

- NHTSA opens third 2025 probe into Tesla Full Self-Driving over red-light runs, lane incursions - [Ars Technica](https://arstechnica.com/cars/2025/10/tesla-fsd-gets-worse-at-driving-nhtsa-opens-new-investigation/)
- Tesla launches Robotaxi in Austin and Bay Area with safety drivers; long waits, spotty availability - [Fast Company](https://www.fastcompany.com/91419058/tesla-robotaxi-bay-area)
- Experts warn of humanoid robot investment bubble; practical humanoids likely a decade away - [TechCrunch](https://techcrunch.com/2025/10/10/the-world-is-just-not-quite-ready-for-humanoids-yet/)

## Other News

- Disney forms cross-company AI and mixed reality unit for films, TV and parks - [Yahoo](https://finance.yahoo.com/news/walt-disney-company-dis-embracing-071526911.html)
- Dialpad launches agentic AI to autonomously resolve up to 70% of customer requests - [The Letter Two](https://thelettertwo.com/2025/10/09/dialpad-agentic-ai-70-percent-customer-requests/)
- Three-quarters of Canadian students use genAI, report better grades but worry about cheating - [CBC](https://www.cbc.ca/news/canada/psstudents-ai-2025-9.6932319)
- Businesses explore ‘empathetic’ agentic AI, stressing transparency, consent and human oversight - [PYMNTS](https://www.pymnts.com/artificial-intelligence-2/2025/businesses-turn-to-empathy-as-ais-new-differentiator/)
- AI agents could gatekeep jobs and deals without verifiable reputation traces by 2027 - [Hacker Noon](https://hackernoon.com/the-ai-gatekeeper-problem-why-your-reputation-needs-to-exist-in-2025)
- Pope warns against AI replacing human judgment in news, urges support for agencies - [International Business Times](https://www.ibtimes.com/pope-hails-role-news-agencies-post-truth-ai-world-3786315)
- Sora 2 skews to teen boys with mocking celebrity cameos, amplifying moderation risks - [Business Insider](https://www.businessinsider.com/sora-videos-openai-content-moderation-celebrities-teens-boys-girls-2025-10)

In [29]:
HTML(newsletter_content)

In [31]:
%pip install markdown 
from utilities import send_gmail
import markdown

Note: you may need to restart the kernel to use updated packages.


In [32]:
newsletter_content_html = markdown.markdown(newsletter_content)

In [33]:
                today = datetime.now().strftime("%B %d, %Y")
                subject = f"AI News Digest - {today}"

                # Apply HTML styling
                html_content = f"""
                <div style="max-width: 800px; margin: 0 auto; font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, 'Helvetica Neue', Arial, sans-serif;">
                    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 40px 20px; text-align: center; border-radius: 8px 8px 0 0;">
                        <h1 style="color: white; margin: 0; font-size: 32px;">AI News Digest</h1>
                        a<p style="color: rgba(255,255,255,0.9); margin: 10px 0 0 0; font-size: 16px;">{today}</p>
                    </div>
                    <div style="background: #ffffff; padding: 30px; border-radius: 0 0 8px 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
                        {newsletter_content_html}
                    </div>
                    <div style="text-align: center; padding: 20px; color: #666; font-size: 14px;">
                        <p>Generated on {today} by AI Newsletter Agent</p>
                    </div>
                </div>
                """

                send_gmail(subject, html_content)

In [ ]:
use mmr 
do the pruning 
rewrite other after pruning
need to pass thinking effort
critic optimizer loop

In [ ]:
input_text = headline_df.loc[headline_df["cat"]=="AI Business Value Gap"].sort_values("rating", ascending=False)[["rating", "short_summary", "site_name", "url"]].to_json(orient="records")
input_text


In [ ]:
write_section_system_prompt, write_section_user_prompt, model = \
    LangfuseClient().get_prompt("newsagent/write_section")

write_section_agent = LLMagent(
    system_prompt=write_section_system_prompt,
    user_prompt=write_section_user_prompt,
    output_type=Section,
    model=model,
    verbose=True,
    logger=logger
)


In [ ]:
response = await write_section_agent.run_prompt(input_text=input_text)
response

In [ ]:
display(Markdown(str(response)))

In [ ]:
class Link(BaseModel):
    url: str = Field(description="URL of the article")
    site_name: str = Field(description="Name of the website/source")
    def __str__(self):
        return f"[{self.site_name}]({self.url})"


class Story(BaseModel):
    headline: str = Field(description="Summary of the story")
    links: List[Link] = Field(description="List of links related to this story")
    prune: bool = Field(description="Whether to prune/exclude this story")
    def __str__(self):
        return "" if self.prune else f"- {self.headline} - " + " ".join([str(s) for s in self.links])


class Section(BaseModel):
    section_title: str = Field(description="Title of the newsletter section")
    headlines: List[Story] = Field(description="List of stories in this section")
    def __str__(self):
        return f"## {self.section_title}\n\n" + "\n".join(
            [str(h) for h in self.headlines]
        )
        

In [ ]:
headline_df=state.headline_df
headline_df.loc[headline_df["cat"]=="AI Business Value Gap"].sort_values("rating", ascending=False)[["rating", "short_summary", "site_name", "url"]].to_json(orient="records")


In [ ]:
print("# SUGGESTED TOPICS:")
catcount = headline_df.groupby("cat").count().reset_index()[['cat', 'source']].sort_values('source', ascending=False)
for c in catcount["cat"]:
    print(c)
print()
print("# RAW NEWS ITEMS:")
i =0
for row in headline_df.sort_values(["cat", "rating"], ascending=False).itertuples():
    print(f"[{row.title}]({row.url}) - {row.site_name}\n".replace("$","\\\\$"))
    row_topics = ", ".join(row.topics)
    print(f"Topics: {row_topics}\n".replace("$","\\\\$"))
    print(f"Rating: {row.rating:.1f}\n")    
    print(f"{row.short_summary}\n".replace("$","\\\\$"))
    print(f"{row.summary}\n".replace("$","\\\\$"))
    print("~~~\n")
    i +=1


In [ ]:
headline_df['cat'].unique() 


In [ ]:
astate.get_completed_steps() 



In [ ]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [ ]:
headline_df=state.headline_df
i =0
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"{row.rating:.1f}"))   
    display(Markdown(f"[{row.title}]({row.url}) - {row.site_name}".replace("$","\\\\$")))
    row_topics = ", ".join(row.topics)
    display(Markdown(f"Topics: {row_topics}".replace("$","\\\\$")))
    display(Markdown(f"{row.short_summary}".replace("$","\\\\$")))
    display(Markdown(f"{row.summary}".replace("$","\\\\$")))
    i +=1
#     if i>=30:
#         break

In [ ]:
class DistilledStory(BaseModel):
    """DistilledStory class for structured output distillation into a single sentence """
    item: str = Field(description="List of StoryRating")
        
system, user, model = LangfuseClient().get_prompt("newsagent/item_distiller")

distill_agent = LLMagent(
            system_prompt=system,
            user_prompt=user,
            output_type=DistilledStory,
            model=model,
            verbose=False,
            logger=logger
        )

In [ ]:
response = await distill_agent.run_prompt(input_text="""AI 'Homeless Man' Challenge Sparks Outrage as Police Called Over Dangerous Viral Trend - International Business Times

Topics: AI Pranks, Public Safety, Ethical Concerns, Disinformation, Policy And Regulation, Snapchat Challenges, Gen AI

Rating: 1.5

Topics: AI Pranks, Public Safety, Ethical Concerns, Disinformation, Policy And Regulation, Snapchat Challenges, Gen AI

Parents in shock after dangerous 'homeless man' Snapchat AI prank goes horribly wrong, police called to calm the viral chaos.

A viral AI-driven prank called the 'homeless man' challenge on Snapchat deceived parents into thinking a homeless person had broken into their home, prompting police intervention.
The prank caused significant public backlash due to the misuse of AI technology to create real emergency scares, leading to police questioning the pranksters and debates about the legal consequences.
Experts emphasize the ethical concerns and potential emotional harm caused by such digital pranks, highlighting the need for responsible use of AI to avoid wasting emergency resources and creating community panic.
""")
response

In [ ]:
headline_df=state.headline_df
i =0
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    print(f"[{row.title}]({row.url}) - {row.site_name}\n".replace("$","\\\\$"))
    row_topics = ", ".join(row.topics)
    print(f"Topics: {row_topics}\n".replace("$","\\\\$"))
    print(f"{row.short_summary}\n".replace("$","\\\\$"))
    print(f"{row.summary}\n".replace("$","\\\\$"))
    print("~~~\n")

    i +=1

TODO:
- update final prompt
- output sections using short summary
# SUGGESTED TOPICS
AI Agents And Reliability
AI Creative Industry Impact
AI Development Tools And Standards
AI Market Valuations
AI Phishing Surge
AI Security Risks
AI Workforce Impact
C2PA Image Provenance
Circular Deal Inflation
Cross-Industry AI Adoption
Crunch Lab Decentralized AI
Data Center Environmental Impact
Deepfake Video Ethics
Dell Raises AI Forecasts
EU AI Strategy
Enterprise AI Data Leakage
Enterprise AI Partnerships
Google Gemini 2.5
Healthcare AI Investments
OpenAI Platform Issues
Other
Qualcomm Acquires Arduino
SoftBank Acquires ABB Robotics
Youth Support AI Ethics

- take each summary and boil it down to 1 sentence , output correct format
- initial write sections - prompt and output json for each section asynchronously
- check and rewrite each section for format asynchronsously
- assemble sections
- do a critic loop